# Imports

In [ ]:
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np
import folium
import pycountry
from shapely import wkt
import networkx as nx
import dill
from mpl_toolkits.basemap import Basemap
import geopandas as gpd
import ast
import utils_pre_processing as upp
from utils_pre_processing import split_col_to_and_from
# Show all columns when using .head()
pd.set_option('display.max_columns', None)

# Reading data

In [ ]:
iggielgn_borderpoints = pd.read_csv('IGGIELGN/IGGIELGN_BorderPoints.csv', delimiter=';')
iggielgn_compressors = pd.read_csv('IGGIELGN/IGGIELGN_Compressors.csv', delimiter=';')
iggielgn_consumers = pd.read_csv('IGGIELGN/IGGIELGN_Consumers.csv', delimiter=';')
iggielgn_lngs = pd.read_csv('IGGIELGN/IGGIELGN_LNGs.csv', delimiter=';')
iggielgn_nodes = pd.read_csv('IGGIELGN/IGGIELGN_Nodes.csv', delimiter=';')
iggielgn_pipesegments = pd.read_csv('IGGIELGN/IGGIELGN_PipeSegments.csv', delimiter=';')
iggielgn_productions = pd.read_csv('IGGIELGN/IGGIELGN_Productions.csv', delimiter=';')
iggielgn_storages = pd.read_csv('IGGIELGN/IGGIELGN_Storages.csv', delimiter=';')

# Pre-processing columns

## Pipe Segments

In [ ]:
# Expanding param, uncertainty and method columns
iggielgn_pipesegments = upp.split_column_to_multiple(iggielgn_pipesegments, 'param')
iggielgn_pipesegments = upp.split_column_to_multiple(iggielgn_pipesegments, 'uncertainty', 'uncertainty_')
iggielgn_pipesegments = upp.split_column_to_multiple(iggielgn_pipesegments, 'method', 'method_')

# Splitting node_id, country_code, lat and long columns
iggielgn_pipesegments = upp.split_col_to_and_from(iggielgn_pipesegments, 'node_id', 'str')
iggielgn_pipesegments = upp.split_col_to_and_from(iggielgn_pipesegments, 'country_code', 'str')
iggielgn_pipesegments = upp.split_coords(iggielgn_pipesegments, 'lat')
iggielgn_pipesegments = upp.split_coords(iggielgn_pipesegments, 'long')

## Border Points

In [ ]:
# Convert values in the node_id col of iggielgn_borderpoints from ['INET_CS_0'] to INET_CS_0. It is a string, so you need to evaluate it first
iggielgn_borderpoints['node_id'] = iggielgn_borderpoints['node_id'].apply(lambda x: ast.literal_eval(x)[0])

# Rename the country_code column to country_code_node
iggielgn_borderpoints.rename(columns={'country_code': 'country_code_node'}, inplace=True)

# Expanding param, uncertainty and method columns
iggielgn_borderpoints = upp.split_column_to_multiple(iggielgn_borderpoints, 'param')
iggielgn_borderpoints = upp.split_column_to_multiple(iggielgn_borderpoints, 'uncertainty', 'uncertainty_')
iggielgn_borderpoints = upp.split_column_to_multiple(iggielgn_borderpoints, 'method', 'method_')

## Compressors

In [ ]:
# Convert values in the node_id col of iggielgn_compressors from ['INET_CS_0'] to INET_CS_0. It is a string, so you need to evaluate it first
iggielgn_compressors['node_id'] = iggielgn_compressors['node_id'].apply(lambda x: ast.literal_eval(x)[0])

# Expanding param, uncertainty and method columns
iggielgn_compressors = upp.split_column_to_multiple(iggielgn_compressors, 'param')
iggielgn_compressors = upp.split_column_to_multiple(iggielgn_compressors, 'uncertainty', 'uncertainty_')
iggielgn_compressors = upp.split_column_to_multiple(iggielgn_compressors, 'method', 'method_')

## LNGs

In [ ]:
# Convert values in the node_id col of iggielgn_lngs from ['INET_CS_0'] to INET_CS_0. It is a string, so you need to evaluate it first
iggielgn_lngs['node_id'] = iggielgn_lngs['node_id'].apply(lambda x: ast.literal_eval(x)[0])

# Expanding param, uncertainty and method columns
iggielgn_lngs = upp.split_column_to_multiple(iggielgn_lngs, 'param')
iggielgn_lngs = upp.split_column_to_multiple(iggielgn_lngs, 'uncertainty', 'uncertainty_')
iggielgn_lngs = upp.split_column_to_multiple(iggielgn_lngs, 'method', 'method_')

## Consumers

In [ ]:
# Convert values in the node_id col of iggielgn_consumers from ['INET_CS_0'] to INET_CS_0. It is a string, so you need to evaluate it first
iggielgn_consumers['node_id'] = iggielgn_consumers['node_id'].apply(lambda x: ast.literal_eval(x)[0])

# Expanding param, uncertainty and method columns
iggielgn_consumers = upp.split_column_to_multiple(iggielgn_consumers, 'param')
iggielgn_consumers = upp.split_column_to_multiple(iggielgn_consumers, 'uncertainty', 'uncertainty_')
iggielgn_consumers = upp.split_column_to_multiple(iggielgn_consumers, 'method', 'method_')

## Productions

In [ ]:
# Convert values in the node_id col of iggielgn_productions from ['INET_CS_0'] to INET_CS_0. It is a string, so you need to evaluate it first
iggielgn_productions['node_id'] = iggielgn_productions['node_id'].apply(lambda x: ast.literal_eval(x)[0])

# Expanding param, uncertainty and method columns
iggielgn_productions = upp.split_column_to_multiple(iggielgn_productions, 'param')
iggielgn_productions = upp.split_column_to_multiple(iggielgn_productions, 'uncertainty', 'uncertainty_')
iggielgn_productions = upp.split_column_to_multiple(iggielgn_productions, 'method', 'method_')

## Storages

In [ ]:
# Convert values in the node_id col of iggielgn_storages from ['INET_CS_0'] to INET_CS_0. It is a string, so you need to evaluate it first
iggielgn_storages['node_id'] = iggielgn_storages['node_id'].apply(lambda x: ast.literal_eval(x)[0])

# Expanding param, uncertainty and method columns
iggielgn_storages = upp.split_column_to_multiple(iggielgn_storages, 'param')
iggielgn_storages = upp.split_column_to_multiple(iggielgn_storages, 'uncertainty', 'uncertainty_')
iggielgn_storages = upp.split_column_to_multiple(iggielgn_storages, 'method', 'method_')

## Nodes

In [ ]:
# Expanding param, uncertainty and method columns
iggielgn_nodes = upp.split_column_to_multiple(iggielgn_nodes, 'param')
iggielgn_nodes = upp.split_column_to_multiple(iggielgn_nodes, 'uncertainty', 'uncertainty_')
iggielgn_nodes = upp.split_column_to_multiple(iggielgn_nodes, 'method', 'method_')

# Add a type column to iggielgn_nodes
iggielgn_nodes['type'] = ''

# Populate the type column of iggielgn_consumers, based on whether the id of the node is in the node_id column of iggielgn_consumers, iggielgn_productions, or iggielgn_storages
for row in iggielgn_nodes.itertuples():
    if row.id in iggielgn_consumers['node_id'].values:
        iggielgn_nodes.loc[iggielgn_nodes['id'] == row.id, 'type'] = 'consumer'
    elif row.id in iggielgn_productions['node_id'].values:
        iggielgn_nodes.loc[iggielgn_nodes['id'] == row.id, 'type'] = 'production'
    elif row.id in iggielgn_storages['node_id'].values:
        iggielgn_nodes.loc[iggielgn_nodes['id'] == row.id, 'type'] = 'storage'
    elif row.id in iggielgn_compressors['node_id'].values:
        iggielgn_nodes.loc[iggielgn_nodes['id'] == row.id, 'type'] = 'compressor'
    elif row.id in iggielgn_lngs['node_id'].values:
        iggielgn_nodes.loc[iggielgn_nodes['id'] == row.id, 'type'] = 'lng'
    elif row.id in iggielgn_borderpoints['node_id'].values:
        iggielgn_nodes.loc[iggielgn_nodes['id'] == row.id, 'type'] = 'borderpoint'
    else:
        iggielgn_nodes.loc[iggielgn_nodes['id'] == row.id, 'type'] = 'node'

In [ ]:
# Plot the nodes on the map
fig, ax = plt.subplots(figsize=(16, 16))

# Use BaseMap to underlay a map of Europe
m = Basemap(projection='merc', llcrnrlat=30, urcrnrlat=70, llcrnrlon=-15, urcrnrlon=80, resolution='l')
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='lightgray')

# Plot the nodes, and color based on the type
for row in iggielgn_nodes.itertuples():
    x, y = m(row.long, row.lat)
    if row.type == 'consumer':
        m.plot(x, y, 'ro', markersize=2)
    elif row.type == 'production':
        m.plot(x, y, 'go', markersize=2)
    elif row.type == 'storage':
        m.plot(x, y, 'bo', markersize=2)
    elif row.type == 'compressor':
        m.plot(x, y, 'yo', markersize=2)
    elif row.type == 'lng':
        m.plot(x, y, 'mo', markersize=2)
    elif row.type == 'borderpoint':
        m.plot(x, y, 'co', markersize=2)
    else:
        m.plot(x, y, 'ko', markersize=2)

# Add legend
plt.plot([], [], 'ro', label='Consumer')
plt.plot([], [], 'go', label='Production')
plt.plot([], [], 'bo', label='Storage')
plt.plot([], [], 'yo', label='Compressor')
plt.plot([], [], 'mo', label='LNG')
plt.plot([], [], 'co', label='Borderpoint')
plt.plot([], [], 'ko', label='Node')
plt.legend(loc='upper right', fontsize=20)

plt.show()

In [ ]:
# Create a networkx graph using the nodes_df as nodes and the pipe_segments_df as edges
G_multi_directed = nx.MultiDiGraph()

# Add nodes to the graph
for index, row in iggielgn_nodes.iterrows():
    # Get the columns of the row, and add the columns as attributes to the node
    attributes = dict(row)
    G_multi_directed.add_node(row['id'], **attributes)

# Add edges to the graph
for index, row in iggielgn_pipesegments.iterrows():
    # Get the columns of the row, and add the columns as attributes to the edge
    attributes = dict(row)
    G_multi_directed.add_edge(row['from_node_id'], row['to_node_id'], **attributes)

# Iterate over the edges once more: if the is_bothDirection is 1.0, add the edge in the opposite direction
for index, row in iggielgn_pipesegments.iterrows():
    if row['is_bothDirection'] == 1.0:
        G_multi_directed.add_edge(row['to_node_id'], row['from_node_id'], **dict(row))

In [ ]:
# Create a simple directed graph, a multi undirected graph, and a simple undirected graph from the directed multigraph

# Aggregating the edges with the same from_node_id and to_node_id, and aggregating the capacity attribute of the edges
G_simple_directed = nx.DiGraph()
for from_node in G_multi_directed.nodes:
    for to_node in G_multi_directed.nodes:
        if G_multi_directed.has_edge(from_node, to_node):
            edge_attrs = G_multi_directed[from_node][to_node][0]
            print(edge_attrs)
            if G_simple_directed.has_edge(from_node, to_node):
                G_simple_directed[from_node][to_node]['max_cap_M_m3_per_d'] += edge_attrs['max_cap_M_m3_per_d']
            else:
                G_simple_directed.add_node(from_node, **G_multi_directed.nodes[from_node])
                G_simple_directed.add_node(to_node, **G_multi_directed.nodes[to_node])
                G_simple_directed.add_edge(from_node, to_node, name=edge_attrs['name'], max_cap_M_m3_per_d=edge_attrs['max_cap_M_m3_per_d'], length_km=edge_attrs['length_km'], diameter_mm=edge_attrs['diameter_mm'], is_H_gas=edge_attrs['is_H_gas'], is_bothDirection=edge_attrs['is_bothDirection'], start_year=edge_attrs['start_year'], end_year=edge_attrs['end_year'], from_country_code=edge_attrs['from_country_code'], to_country_code=edge_attrs['to_country_code'])

G_multi_undirected = nx.MultiGraph(G_multi_directed)
G_simple_undirected = nx.Graph(G_multi_directed)

In [ ]:
# Saving the multi di-graph to a pickle file
with open('graph_objects/G_multi_directed_iggielgn.pickle', 'wb') as file:
    dill.dump(G_multi_directed, file)

# Saving the simple di-graph to a pickle file
with open('graph_objects/G_simple_directed_iggielgn.pickle', 'wb') as file:
    dill.dump(G_simple_directed, file)

# Saving the multi un-graph to a pickle file
with open('graph_objects/G_multi_undirected_iggielgn.pickle', 'wb') as file:
    dill.dump(G_multi_undirected, file)

# Saving the simple un-graph to a pickle file
with open('graph_objects/G_simple_undirected_iggielgn.pickle', 'wb') as file:
    dill.dump(G_simple_undirected, file)

In [ ]:
# Print the number of edges and nodes in each of the four graphs
print("Number of edges and nodes in the four graphs:")
print("G:", G_multi_directed.number_of_edges(), G_multi_directed.number_of_nodes())
print("G_simple_directed:", G_simple_directed.number_of_edges(), G_simple_directed.number_of_nodes())
print("G_multi_undirected:", G_multi_undirected.number_of_edges(), G_multi_undirected.number_of_nodes())
print("G_simple_undirected:", G_simple_undirected.number_of_edges(), G_simple_undirected.number_of_nodes())

In [ ]:
# Plot the graph
fig, ax = plt.subplots(figsize=(16, 16))

# Use BaseMap to underlay a map of Europe
m = Basemap(projection='merc', llcrnrlat=30, urcrnrlat=70, llcrnrlon=-15, urcrnrlon=80, resolution='l')
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='lightgray')

# Plot the edges
for edge in G_multi_directed.edges():
    node1 = G_multi_directed.nodes[edge[0]]
    node2 = G_multi_directed.nodes[edge[1]]
    x1, y1 = m(node1['long'], node1['lat'])
    x2, y2 = m(node2['long'], node2['lat'])
    m.plot([x1, x2], [y1, y2], 'k-')


# Add legend
plt.legend(loc='upper right', fontsize=20)

plt.show()

In [ ]:
# Ouput the connected components of the graph
connected_components = list(nx.weakly_connected_components(G_multi_directed))
connected_components_sizes = [len(comp) for comp in connected_components]
connected_components_sizes.sort(reverse=True)
print(connected_components_sizes)

In [ ]:
# Only plot nodes not in the largest connected component
largest_component = connected_components[0]
non_largest_component_nodes = [node for node in G_multi_directed.nodes() if node not in largest_component]

# Plot the nodes on the map, and color based on the component id
fig, ax = plt.subplots(figsize=(16, 16))

# Use BaseMap to underlay a map of Europe
m = Basemap(projection='merc', llcrnrlat=30, urcrnrlat=70, llcrnrlon=-15, urcrnrlon=80, resolution='l')
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color='lightgray')

# Plot the nodes, and color based on the component id
for row in iggielgn_nodes.itertuples():
    if row.id in non_largest_component_nodes:
        x, y = m(row.long, row.lat)
        m.plot(x, y, 'ko', markersize=2)

plt.show()

In [ ]:
# Plot a histogram for the is_bothDirection in the iggielgn_pipesegments dataframe
iggielgn_pipesegments['is_bothDirection'].hist()

In [ ]:
# Output the data of one edge in the graph
print(list(G_multi_directed.nodes(data=True))[0])
print(list(G_multi_directed.edges(data=True))[0])

In [ ]:
# Show the degree distribution of the graph
degrees = [G_multi_directed.degree(node) for node in G_multi_directed.nodes()]
plt.hist(degrees)
plt.show()

In [ ]:
# Plot the in- and out-degree distribution for each node type using a bar chart, where the x-asis it the degree and the y-axis is the number of nodes with that degree
node_types = iggielgn_nodes['type'].unique()
for node_type in node_types:
    nodes_of_type = iggielgn_nodes[iggielgn_nodes['type'] == node_type]
    in_degrees = [G_multi_directed.in_degree(node) for node in nodes_of_type['id']]
    out_degrees = [G_multi_directed.out_degree(node) for node in nodes_of_type['id']]
    plt.figure(figsize=(16, 8))
    plt.subplot(1, 2, 1)
    plt.hist(in_degrees, bins=20)
    plt.title(f'In-degree distribution for {node_type}')
    plt.xlabel('In-degree')
    plt.ylabel('Number of nodes')
    plt.subplot(1, 2, 2)
    plt.hist(out_degrees, bins=20)
    plt.title(f'Out-degree distribution for {node_type}')
    plt.xlabel('Out-degree')
    plt.ylabel('Number of nodes')
    plt.show()
